## 2021 빅데이터 플랫폼 경진대회
  
**[접근]**음식점 창업... 해도 될까요?!
- 어느 업종이 좋을까?
- 어느 지역이 좋을까?
- 지자체는 자영업자들의 생존을 위해 무엇을 지원해야 할까?  
  
배달 데이터: 
> 기간: **2019-12-01 ~ 2020-05-31**  

행정동별 생활인구 데이터:  
> 행정동코드: **행정동코드**를 이용하여 집계  

휴/폐업 데이터:  
> 상호별, 지역별 휴/페업 정보 존재: 지역별 휴/폐업 정보에서 **위/경도를 사용**하여 추후 공간 시각화  

#### 

## 내가할일은?? <공간 시각화를 진행해보자>
1) 지역별 휴/폐업 위치  
> 휴/폐업이 많은 지역은 어디?

2) 배달상점ID별 주문건수 집계 후 시각화  
> 주문이 많이 들어온 상점 위치는?  

3) 배달목적지별 주문건수 집계 후 시각화  
> 주문을 많이 하는 지역은?

####  

### 00. Package Load  

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")
import time

import folium

#### 

### 01. Data Load 

* **r'' 의미**  
Raw string이라고 해서, "순수한 문자"임을 알려주는 문법. '\n' 등과 같은경우 줄바꿈이 작용하기 때문에 이를 순수한 문자로 인식시키기 위해 사용

In [2]:
kt_data_path = r'/Users/lion_oh/Desktop/hr/project/경진대회/교통데이터 공모전/code/dataset/kt_dataset/'
covid19_path = r'/Users/lion_oh/Desktop/hr/project/경진대회/교통데이터 공모전/code/dataset/covid_19_dataset/'
geo_path = r'/Users/lion_oh/Desktop/hr/project/경진대회/교통데이터 공모전/code/dataset/geo_dataset/'

In [3]:
kt_file_list = os.listdir(kt_data_path)
display(kt_file_list)

covid_file_list = os.listdir(covid19_path)
display(covid_file_list)

geo_file_list = os.listdir(geo_path)
display(geo_file_list)

['서울시_직장인구.csv',
 'StartupOrClosing_Shop.csv',
 '서울시_동별_1인가구.txt',
 'dlvr_call.csv',
 'StartupOrClosing_Region.csv',
 '전체 데이터 설명.hwp',
 'popl.csv']

['SeoulFloating.csv',
 'TimeAge.csv',
 'SearchTrend.csv',
 'TimeProvince.csv',
 'Weather.csv',
 'PatientInfo.csv',
 'Region.csv',
 'TimeGender.csv',
 'Policy.csv',
 'Case.csv',
 'Time.csv']

['HangJeongDong_ver20210401.geojson',
 '.DS_Store',
 'seoul_municipalities_geo.json.txt']

#### 

### 02. 서울시 휴/폐업이 많은 행정동은 어디??
1) 창/폐업 시각화  
2) 동단위별 집계 후, Top5 창/폐업 행정동 시각화  
3) 월별 Top5 창/폐업 행정동은??  

In [8]:
kt_file_list[2]
region_cols = ['인덱스','날짜','업종명','광역시도명','시군구명','읍면동','위도값','경도값','구분']
sc_region = pd.read_csv(kt_data_path + kt_file_list[4], header = 0, encoding = "cp949", names = region_cols)
sc_region.head()

인덱스        날짜       업종명 광역시도명       시군구명   읍면동        위도값         경도값  구분
0    1  20200101      고등학교    서울        관악구   봉천동  37.487***  126.943***   1
1    2  20200101    노인보호센터    전남    순천시 상사면   오곡리  34.909***  127.461***   1
2    3  20200101  노인생활복지시설    강원       강릉시    용강동  37.754***  128.890***   1
3    4  20200101      돼지국밥    경남  창원시 마산합포구    현동  35.147***  128.557***   1
4    5  20200101    레저용품대여    전북       군산시   비응도동  35.939***  126.529***   1

In [9]:
sc_region.columns

Index(['인덱스', '날짜', '업종명', '광역시도명', '시군구명', '읍면동', '위도값', '경도값', '구분'], dtype='object')

In [16]:
# 업종명에 맞춰서 배달상점업종명을 만들어줍니다.
# 정말 힘들었습니다.

sc_region.loc[sc_region.업종명.isin(['한식', '육류.고기요리', '칼국수', '곱창.양구이', '오리고기', '냉면', '장어',
                            '한정식', '해물.생선요리', '닭갈비', '국수', '삼겹살', '돼지갈비', '게요리',
                            '낙지요리', '오징어요리', '한우고기', '쭈꾸미요리', '막국수', '두부요리', '전복요리',
                            '쌈밥', '닭요리', '순두부', '보리밥', '비빕밥', '흙돼지', '코다리전문', '오리요리',
                            '음식점-닭요리전문', '닭집', '밀면', '콩국수', '한식배달', '비빕밥', '굴요리']),
           '배달상점업종명'] = '한식'

sc_region.loc[sc_region.업종명.isin(['감자탕', '추어탕', '사철.영양탕', '순대.순대국', '부대찌개', '음식점-생태/동태/명태요리전문',
                            '설렁탕', '삼계탕', '해장국', '샤브샤브', '곰탕', '소머리국밥', '매운탕', '찌개.전골', '찜닭',
                            '돼지국밥', '어죽', '순대국밥', '백숙', '붕어찜', '소고기국밥', '버섯요리', '대구탕', '어죽',
                            '아귀요리', '복어요리', '조개구이']),
           '배달상점업종명'] = '찜탕'

sc_region.loc[sc_region.업종명.isin(['카페', '제과점', '떡집', '음료제조(종합)', '빵.과자', '떡.한과', '핫도그', '샌드위치',
                            '카페-디저트', '토스트', '전통찻집', '호두과자', '도너츠', '감자튀김', '와플전문점',
                           '케이크전문점', '오메기떡판매', '수제파이', '보리빵']),
           '배달상점업종명'] = '카페/디저트'

sc_region.loc[sc_region.업종명.isin(['양식', '베트남음식', '패밀리레스토랑', '퓨전음식', '태국음식', '양고기',
                            '스파게티', '이탈리아요리', '인도음식', '스페인요리', '멕시코요리', '동남아음식']),
           '배달상점업종명'] = '아시안/양식'

sc_region.loc[sc_region.업종명.isin(['일식', '참치요리', '돈가스', '일식튀김.꼬치', '우동', '초밥.롤', '철판구이']),
           '배달상점업종명'] = '일식'

sc_region.loc[sc_region.업종명.isin(['횟집', '홍어', '수산물음식점', '문어요리']),
           '배달상점업종명'] = '회'

sc_region.loc[sc_region.업종명.isin(['분식', '김밥', '떡볶이', '라면', '주먹밥']),
           '배달상점업종명'] = '분식'

sc_region.loc[sc_region.업종명.isin(['중국음식', '음식점-마라탕', '만두']), '배달상점업종명'] = '중식'
sc_region.loc[sc_region.업종명.isin(['치킨', '닭강정']), '배달상점업종명'] = '치킨'
sc_region.loc[sc_region.업종명.isin(['패스트푸드(종합)', '햄버거']), '배달상점업종명'] = '패스트푸드'
sc_region.loc[sc_region.업종명.isin(['닭발', '야식']), '배달상점업종명'] = '야식'
sc_region.loc[sc_region.업종명.isin(['피자']), '배달상점업종명'] = '피자'
sc_region.loc[sc_region.업종명.isin(['도시락']), '배달상점업종명'] = '도시락'
sc_region.loc[sc_region.업종명.isin(['족발.보쌈']), '배달상점업종명'] = '족발/보쌈'

In [17]:
sc_region.head()

인덱스        날짜       업종명 광역시도명       시군구명   읍면동        위도값         경도값  구분  \
0    1  20200101      고등학교    서울        관악구   봉천동  37.487***  126.943***   1   
1    2  20200101    노인보호센터    전남    순천시 상사면   오곡리  34.909***  127.461***   1   
2    3  20200101  노인생활복지시설    강원       강릉시    용강동  37.754***  128.890***   1   
3    4  20200101      돼지국밥    경남  창원시 마산합포구    현동  35.147***  128.557***   1   
4    5  20200101    레저용품대여    전북       군산시   비응도동  35.939***  126.529***   1   

  배달상점업종명  
0     NaN  
1     NaN  
2     NaN  
3      찜탕  
4     NaN

In [18]:
sc_region_food = sc_region[~sc_region["배달상점업종명"].isnull()].reset_index(drop = True)
sc_region_food

인덱스        날짜   업종명 광역시도명       시군구명   읍면동        위도값         경도값  \
0          4  20200101  돼지국밥    경남  창원시 마산합포구    현동  35.147***  128.557***   
1         11  20200101    한식    강원    영월군 영월읍   영흥리  37.187***  128.473***   
2         16  20200102   삼계탕    제주    제주시 조천읍   함덕리  33.543***  126.655***   
3         18  20200102  스파게티   경기도       부천시     중동  37.501***  126.775***   
4         22  20200102  중국음식    광주         북구   용두동  35.213***  126.876***   
...      ...       ...   ...   ...        ...   ...        ...         ...   
10174  87845  20200630    횟집    경남    창원시 성산구   가음동  35.205***  128.699***   
10175  87846  20200630    횟집    경남    진주시 대곡면   광석리  35.262***  128.168***   
10176  87847  20200630    횟집    경북     포항시 북구   환호동  36.069***  129.400***   
10177  87848  20200630    횟집    대구        수성구  범어4동  35.858***  128.640***   
10178  87849  20200630    횟집    광주         북구   우산동  35.166***  126.916***   

       구분 배달상점업종명  
0       1      찜탕  
1       1      한식  
2       1      찜탕  
3       0  아시안/양식  
4       1      중식  
...    ..     ...  
10174   0       회  
10175   0       회  
10176   0       회  
10177   0       회  
10178   0       회  

[10179 rows x 10 columns]

In [20]:
## 서울만 뽑아서 확인
sc_region_seoul = sc_region_food[sc_region_food["광역시도명"] == '서울'].reset_index(drop = True)
sc_region_seoul

인덱스        날짜     업종명 광역시도명 시군구명   읍면동        위도값         경도값  구분  \
0        29  20200103      카페    서울  마포구   상암동  37.580***  126.889***   0   
1        97  20200109  카페-디저트    서울  종로구   적선동  37.574***  126.973***   0   
2       166  20200117      한식    서울  서초구   서초동  37.487***  127.013***   1   
3       192  20200120      야식    서울  송파구   방이동  37.512***  127.111***   0   
4       208  20200122      분식    서울  송파구   잠실동  37.506***  127.085***   1   
...     ...       ...     ...   ...  ...   ...        ...         ...  ..   
1191  87711  20200630      피자    서울  금천구  독산본동  37.472***  126.902***   0   
1192  87735  20200630      한식    서울  강북구   수유동  37.634***  127.020***   0   
1193  87760  20200630      한식    서울  서초구   서초동  37.485***  127.012***   0   
1194  87762  20200630      한식    서울  송파구   석촌동  37.499***  127.096***   0   
1195  87773  20200630      한식    서울  성동구   행당동  37.559***  127.040***   0   

     배달상점업종명  
0     카페/디저트  
1     카페/디저트  
2         한식  
3         야식  
4         분식  
...      ...  
1191      피자  
1192      한식  
1193      한식  
1194      한식  
1195      한식  

[1196 rows x 10 columns]

In [21]:
sc_region_seoul.shape

(1196, 10)

In [22]:
def _get_div(div):
    '''
        바이너리 인코딩 되어 있는 창/폐업 정보를 문자열로 표현하는 함수 정의
        div: 창/폐업 바이너리 정보
    '''
    
    if div == 1:
        return "창업"
    else:
        return "폐업"
    
sc_region_seoul["Div_kor"] = sc_region_seoul.apply(lambda x: _get_div(x["구분"]), axis = 1)
sc_region_seoul["Div_kor"]

0       폐업
1       폐업
2       창업
3       폐업
4       창업
        ..
1191    폐업
1192    폐업
1193    폐업
1194    폐업
1195    폐업
Name: Div_kor, Length: 1196, dtype: object

In [23]:
sc_region_seoul.columns

Index(['인덱스', '날짜', '업종명', '광역시도명', '시군구명', '읍면동', '위도값', '경도값', '구분',
       '배달상점업종명', 'Div_kor'],
      dtype='object')

In [24]:
sc_region_seoul["위도값"][0]

'37.580***'

In [25]:
## 마스킹 처리된 위/경도 데이터 전처리
### '***': 제거
### 데이터 타입을 int형으로 변환

sc_region_seoul['위도값'] = sc_region_seoul['위도값'].apply(lambda x: x.replace('*',''))
display(sc_region_seoul['위도값'])

sc_region_seoul['위도값'] = sc_region_seoul['위도값'].astype(float)
display(sc_region_seoul['위도값'])

sc_region_seoul['경도값'] = sc_region_seoul['경도값'].apply(lambda x: x.replace('*',''))
display(sc_region_seoul['경도값'])

sc_region_seoul['경도값'] = sc_region_seoul['경도값'].astype(float)
display(sc_region_seoul['경도값'])

0       37.580
1       37.574
2       37.487
3       37.512
4       37.506
         ...  
1191    37.472
1192    37.634
1193    37.485
1194    37.499
1195    37.559
Name: 위도값, Length: 1196, dtype: object

0       37.580
1       37.574
2       37.487
3       37.512
4       37.506
         ...  
1191    37.472
1192    37.634
1193    37.485
1194    37.499
1195    37.559
Name: 위도값, Length: 1196, dtype: float64

0       126.889
1       126.973
2       127.013
3       127.111
4       127.085
         ...   
1191    126.902
1192    127.020
1193    127.012
1194    127.096
1195    127.040
Name: 경도값, Length: 1196, dtype: object

0       126.889
1       126.973
2       127.013
3       127.111
4       127.085
         ...   
1191    126.902
1192    127.020
1193    127.012
1194    127.096
1195    127.040
Name: 경도값, Length: 1196, dtype: float64

In [26]:
## 공간시각화
### step1: 지도부터 시각화
lat = sc_region_seoul['위도값'].mean()
lon = sc_region_seoul['경도값'].mean()

m = folium.Map([lat, lon], zoom_start = 9)
m

### step2: 지도위에 데이터 찍자
for i in sc_region_seoul.index:
    sub_lat = sc_region_seoul.loc[i, '위도값']
    sub_lon = sc_region_seoul.loc[i, '경도값']
    
    title = sc_region_seoul.loc[i, 'Div_kor']
    
    color = 'green'
    if sc_region_seoul.loc[i, 'Div_kor'] == '폐업':
        color = 'red'
    
    folium.CircleMarker([sub_lat, sub_lon], color=color, radius = 3, tooltip = title).add_to(m)
    
m

In [27]:
# 창업이 많지는 않네...
sc_region_seoul.Div_kor.value_counts()

폐업    1167
창업      29
Name: Div_kor, dtype: int64

In [28]:
sc_region_seoul

인덱스        날짜     업종명 광역시도명 시군구명   읍면동     위도값      경도값  구분 배달상점업종명  \
0        29  20200103      카페    서울  마포구   상암동  37.580  126.889   0  카페/디저트   
1        97  20200109  카페-디저트    서울  종로구   적선동  37.574  126.973   0  카페/디저트   
2       166  20200117      한식    서울  서초구   서초동  37.487  127.013   1      한식   
3       192  20200120      야식    서울  송파구   방이동  37.512  127.111   0      야식   
4       208  20200122      분식    서울  송파구   잠실동  37.506  127.085   1      분식   
...     ...       ...     ...   ...  ...   ...     ...      ...  ..     ...   
1191  87711  20200630      피자    서울  금천구  독산본동  37.472  126.902   0      피자   
1192  87735  20200630      한식    서울  강북구   수유동  37.634  127.020   0      한식   
1193  87760  20200630      한식    서울  서초구   서초동  37.485  127.012   0      한식   
1194  87762  20200630      한식    서울  송파구   석촌동  37.499  127.096   0      한식   
1195  87773  20200630      한식    서울  성동구   행당동  37.559  127.040   0      한식   

     Div_kor  
0         폐업  
1         폐업  
2         창업  
3         폐업  
4         창업  
...      ...  
1191      폐업  
1192      폐업  
1193      폐업  
1194      폐업  
1195      폐업  

[1196 rows x 11 columns]

In [29]:
sc_region_seoul['구분']

0       0
1       0
2       1
3       0
4       1
       ..
1191    0
1192    0
1193    0
1194    0
1195    0
Name: 구분, Length: 1196, dtype: int64

In [30]:
## Top 5 폐업 행정동
top = lambda x: x.sort_values(ascending = False)[:5]
top5_region = sc_region_seoul[sc_region_seoul['구분'] == 0].groupby(["시군구명","읍면동"]).agg({"Div_kor":len}).apply(top).reset_index()
top5_region

시군구명   읍면동  Div_kor
0  영등포구  여의도동       26
1   서초구   서초동       24
2   강남구   역삼동       23
3   강남구   삼성동       22
4   강남구   신사동       19

In [31]:
region = top5_region["읍면동"].to_list()
sc_top5_region = sc_region_seoul[(sc_region_seoul['읍면동'].isin(region)) & (sc_region_seoul['시군구명'] != '은평구')].reset_index(drop = True)
sc_top5_region

인덱스        날짜     업종명 광역시도명  시군구명   읍면동     위도값      경도값  구분 배달상점업종명  \
0      166  20200117      한식    서울   서초구   서초동  37.487  127.013   1      한식   
1      366  20200207  카페-디저트    서울   강남구   역삼동  37.500  127.039   0  카페/디저트   
2      449  20200214    중국음식    서울  영등포구  여의도동  37.530  126.916   0      중식   
3     1218  20200331      일식    서울   강남구   역삼동  37.498  127.031   0      일식   
4     1631  20200522     한정식    서울   강남구   삼성동  37.504  127.025   0      한식   
..     ...       ...     ...   ...   ...   ...     ...      ...  ..     ...   
111  83404  20200623      한식    서울   강남구   역삼동  37.493  127.040   0      한식   
112  86147  20200627    참치요리    서울   강남구   신사동  37.517  127.021   0      일식   
113  86215  20200627      카페    서울   강남구   신사동  37.518  127.023   0  카페/디저트   
114  87507  20200630    중국음식    서울  영등포구  여의도동  37.530  126.921   0      중식   
115  87760  20200630      한식    서울   서초구   서초동  37.485  127.012   0      한식   

    Div_kor  
0        창업  
1        폐업  
2        폐업  
3        폐업  
4        폐업  
..      ...  
111      폐업  
112      폐업  
113      폐업  
114      폐업  
115      폐업  

[116 rows x 11 columns]

In [32]:
### Top 5 페업 행정 공간시각화
### step1: 지도부터 시각화
lat = sc_top5_region['위도값'].mean()
lon = sc_top5_region['경도값'].mean()

m = folium.Map([lat, lon], zoom_start = 9)
m

### step2: 데이터 지도에 찍기
for i in sc_top5_region.index:
    sub_lat = sc_top5_region.loc[i, '위도값']
    sub_lon = sc_top5_region.loc[i, '경도값']
    
    title = sc_top5_region.loc[i, 'Div_kor']
    
    color = 'green'
    if sc_top5_region.loc[i, 'Div_kor'] == '폐업':
        color = 'red'
    
    folium.CircleMarker([sub_lat, sub_lon], color=color, radius = 5, tooltip = title).add_to(m)
    
m

#### 행정동별 전체 요식업 수 대비 폐업 (여기해야함!!!!!)
* 배민 데이터에서 없는 시군구 있음
* folium에서 동단위 색칠한느 법

In [33]:
kt_file_list

['서울시_직장인구.csv',
 'StartupOrClosing_Shop.csv',
 '서울시_동별_1인가구.txt',
 'dlvr_call.csv',
 'StartupOrClosing_Region.csv',
 '전체 데이터 설명.hwp',
 'popl.csv']

In [36]:
dlvr_cols = ['순번', '처리일시', '배달요청업체ID', '배달요청상태값','배달접수업체ID','배달상점ID','배달상점업종명',
             '배달상점법정동코드','배달상점광역시도명','배달상점시군구명','배달상점법정동명','배달상점리명',
             '배달상점행정동코드','배달상점도로명주소코드','배달목적지법정동코드','배달목적지광역시도명','배달목적지시군구명',
             '배달목적지법정동명','배달목적지리명','배달목적지행정동코드','배달목적지도로명주소코드','배달자ID',
             '배달금액','호출중계수수료금액','상품금액','결제종류값','결제카드명','배달접수시간','배달배차시간',
             '배달수령시간','배달완료시간','배달취소시간']

dlvr = pd.read_csv(kt_data_path + kt_file_list[3], header = 0, encoding = "utf-8", names = dlvr_cols)
dlvr.head()

순번        처리일시 배달요청업체ID  배달요청상태값 배달접수업체ID    배달상점ID 배달상점업종명  \
0  3486929  2019-12-01  C000101        1  C000101  S0021950   족발/보쌈   
1  3486930  2019-12-01  C000099        1  C000099  S0039210      치킨   
2  3486932  2019-12-01  C000029        1  C000043  S0036135      치킨   
3  3486933  2019-12-01  C000105        1  C000105  S0039956  돈까스/일식   
4  3486934  2019-12-01  C000028        1  C000043  S0003074      치킨   

    배달상점법정동코드 배달상점광역시도명  배달상점시군구명  ...  배달금액 호출중계수수료금액   상품금액 결제종류값  \
0  1156013200     서울특별시      영등포구  ...  3300       200  18000     1   
1  4128510200       경기도  고양시 일산동구  ...  2900         0  21900     1   
2  4115010100       경기도      의정부시  ...  3500       200  19500     2   
3  4128510400       경기도  고양시 일산동구  ...  3500         0  18000     2   
4  4115010900       경기도      의정부시  ...  3500       200  18000     3   

       결제카드명               배달접수시간               배달배차시간               배달수령시간  \
0  승인:KB국민카드  2019-12-01 20:47:52  2019-12-01 20:48:11  2019-12-01 20:55:33   
1   승인:현대카드사  2019-12-01 21:58:07  2019-12-01 22:01:06  2019-12-01 22:20:53   
2        NaN  2019-12-01 22:20:31  2019-12-01 22:37:45  2019-12-01 22:58:27   
3        NaN  2019-12-01 21:40:39  2019-12-01 21:48:41  2019-12-01 22:06:35   
4        NaN  2019-12-01 19:44:37  2019-12-01 19:48:07  2019-12-01 19:54:30   

                배달완료시간 배달취소시간  
0  2019-12-01 21:00:15     \N  
1  2019-12-01 22:26:01     \N  
2  2019-12-01 22:58:32     \N  
3  2019-12-01 22:17:57     \N  
4  2019-12-01 20:00:22     \N  

[5 rows x 32 columns]

In [37]:
dlvr.columns

Index(['순번', '처리일시', '배달요청업체ID', '배달요청상태값', '배달접수업체ID', '배달상점ID', '배달상점업종명',
       '배달상점법정동코드', '배달상점광역시도명', '배달상점시군구명', '배달상점법정동명', '배달상점리명', '배달상점행정동코드',
       '배달상점도로명주소코드', '배달목적지법정동코드', '배달목적지광역시도명', '배달목적지시군구명', '배달목적지법정동명',
       '배달목적지리명', '배달목적지행정동코드', '배달목적지도로명주소코드', '배달자ID', '배달금액', '호출중계수수료금액',
       '상품금액', '결제종류값', '결제카드명', '배달접수시간', '배달배차시간', '배달수령시간', '배달완료시간',
       '배달취소시간'],
      dtype='object')

In [38]:
dlvr.isnull().sum()

순번                    0
처리일시                  0
배달요청업체ID              0
배달요청상태값               0
배달접수업체ID              0
배달상점ID                0
배달상점업종명               0
배달상점법정동코드             0
배달상점광역시도명             0
배달상점시군구명              0
배달상점법정동명              0
배달상점리명                0
배달상점행정동코드             0
배달상점도로명주소코드           0
배달목적지법정동코드            0
배달목적지광역시도명            0
배달목적지시군구명             0
배달목적지법정동명             0
배달목적지리명               0
배달목적지행정동코드            0
배달목적지도로명주소코드          0
배달자ID                 0
배달금액                  0
호출중계수수료금액             0
상품금액                  0
결제종류값                 0
결제카드명           3177885
배달접수시간                0
배달배차시간                0
배달수령시간                0
배달완료시간                0
배달취소시간                0
dtype: int64

In [39]:
cols = ['배달상점광역시도명', '배달상점시군구명', '배달상점법정동명']
dlvr_1 = dlvr[cols]
dlvr_1 = dlvr_1[dlvr_1["배달상점광역시도명"] == "서울특별시"]
dlvr_1

배달상점광역시도명 배달상점시군구명 배달상점법정동명
0           서울특별시     영등포구      신길동
9           서울특별시      구로구      오류동
17          서울특별시      구로구      고척동
18          서울특별시      금천구      독산동
19          서울특별시      구로구      고척동
...           ...      ...      ...
4296259     서울특별시      은평구      갈현동
4296262     서울특별시      은평구      갈현동
4296265     서울특별시      은평구      갈현동
4296267     서울특별시     영등포구      대림동
4296270     서울특별시      금천구      독산동

[980890 rows x 3 columns]

In [40]:
## 시군구 및 동단위 집계
dlvr_2 = dlvr_1.groupby(["배달상점시군구명", "배달상점법정동명"]).agg(len).reset_index()
dlvr_2

배달상점시군구명 배달상점법정동명  배달상점광역시도명
0       강남구      개포동        269
1       강남구      논현동          1
2       강남구      대치동        313
3       강남구      삼성동       1150
4       강남구      역삼동          1
..      ...      ...        ...
72      은평구      대조동      16264
73      은평구      신사동          7
74      은평구      역촌동       4936
75      중랑구       묵동         10
76      중랑구      중화동          1

[77 rows x 3 columns]

In [41]:
dlvr_2

배달상점시군구명 배달상점법정동명  배달상점광역시도명
0       강남구      개포동        269
1       강남구      논현동          1
2       강남구      대치동        313
3       강남구      삼성동       1150
4       강남구      역삼동          1
..      ...      ...        ...
72      은평구      대조동      16264
73      은평구      신사동          7
74      은평구      역촌동       4936
75      중랑구       묵동         10
76      중랑구      중화동          1

[77 rows x 3 columns]

In [42]:
seoul_dlvr = pd.merge(left = sc_region_seoul, right = dlvr_2, left_on = ['시군구명', '읍면동'], right_on = ['배달상점시군구명', '배달상점법정동명'], how = 'left')
seoul_dlvr

인덱스        날짜     업종명 광역시도명 시군구명   읍면동     위도값      경도값  구분 배달상점업종명  \
0        29  20200103      카페    서울  마포구   상암동  37.580  126.889   0  카페/디저트   
1        97  20200109  카페-디저트    서울  종로구   적선동  37.574  126.973   0  카페/디저트   
2       166  20200117      한식    서울  서초구   서초동  37.487  127.013   1      한식   
3       192  20200120      야식    서울  송파구   방이동  37.512  127.111   0      야식   
4       208  20200122      분식    서울  송파구   잠실동  37.506  127.085   1      분식   
...     ...       ...     ...   ...  ...   ...     ...      ...  ..     ...   
1191  87711  20200630      피자    서울  금천구  독산본동  37.472  126.902   0      피자   
1192  87735  20200630      한식    서울  강북구   수유동  37.634  127.020   0      한식   
1193  87760  20200630      한식    서울  서초구   서초동  37.485  127.012   0      한식   
1194  87762  20200630      한식    서울  송파구   석촌동  37.499  127.096   0      한식   
1195  87773  20200630      한식    서울  성동구   행당동  37.559  127.040   0      한식   

     Div_kor 배달상점시군구명 배달상점법정동명  배달상점광역시도명  
0         폐업      NaN      NaN        NaN  
1         폐업      NaN      NaN        NaN  
2         창업      서초구      서초동       92.0  
3         폐업      NaN      NaN        NaN  
4         창업      송파구      잠실동        2.0  
...      ...      ...      ...        ...  
1191      폐업      NaN      NaN        NaN  
1192      폐업      NaN      NaN        NaN  
1193      폐업      서초구      서초동       92.0  
1194      폐업      NaN      NaN        NaN  
1195      폐업      NaN      NaN        NaN  

[1196 rows x 14 columns]

In [43]:
seoul_dlvr["배달상점광역시도명"] = seoul_dlvr["배달상점광역시도명"].fillna(0)

In [44]:
top = lambda x: x.sort_values(ascending = False)[:5]
seoul_dlvr.groupby(["시군구명","읍면동"]).agg({"배달상점광역시도명":np.max}).apply(top)

배달상점광역시도명
시군구명 읍면동           
구로구  개봉동   151326.0
금천구  독산동    89345.0
구로구  고척동    72805.0
은평구  갈현동    60062.0
금천구  시흥동    59149.0

In [45]:
seoul_dlvr

인덱스        날짜     업종명 광역시도명 시군구명   읍면동     위도값      경도값  구분 배달상점업종명  \
0        29  20200103      카페    서울  마포구   상암동  37.580  126.889   0  카페/디저트   
1        97  20200109  카페-디저트    서울  종로구   적선동  37.574  126.973   0  카페/디저트   
2       166  20200117      한식    서울  서초구   서초동  37.487  127.013   1      한식   
3       192  20200120      야식    서울  송파구   방이동  37.512  127.111   0      야식   
4       208  20200122      분식    서울  송파구   잠실동  37.506  127.085   1      분식   
...     ...       ...     ...   ...  ...   ...     ...      ...  ..     ...   
1191  87711  20200630      피자    서울  금천구  독산본동  37.472  126.902   0      피자   
1192  87735  20200630      한식    서울  강북구   수유동  37.634  127.020   0      한식   
1193  87760  20200630      한식    서울  서초구   서초동  37.485  127.012   0      한식   
1194  87762  20200630      한식    서울  송파구   석촌동  37.499  127.096   0      한식   
1195  87773  20200630      한식    서울  성동구   행당동  37.559  127.040   0      한식   

     Div_kor 배달상점시군구명 배달상점법정동명  배달상점광역시도명  
0         폐업      NaN      NaN        0.0  
1         폐업      NaN      NaN        0.0  
2         창업      서초구      서초동       92.0  
3         폐업      NaN      NaN        0.0  
4         창업      송파구      잠실동        2.0  
...      ...      ...      ...        ...  
1191      폐업      NaN      NaN        0.0  
1192      폐업      NaN      NaN        0.0  
1193      폐업      서초구      서초동       92.0  
1194      폐업      NaN      NaN        0.0  
1195      폐업      NaN      NaN        0.0  

[1196 rows x 14 columns]

In [46]:
seoul_dlvr1 = seoul_dlvr.groupby(["시군구명", "읍면동"]).agg({"배달상점광역시도명":np.max}).reset_index()
seoul_dlvr1

시군구명   읍면동  배달상점광역시도명
0    강남구   개포동      269.0
1    강남구  논현1동        0.0
2    강남구  논현2동        0.0
3    강남구   논현동        1.0
4    강남구  대치2동        0.0
..   ...   ...        ...
410  중랑구  상봉2동        0.0
411  중랑구   상봉동        0.0
412  중랑구   신내동        0.0
413  중랑구  중화2동        0.0
414  중랑구  중화3동        0.0

[415 rows x 3 columns]

In [47]:
geo_path

'/Users/lion_oh/Desktop/hr/project/경진대회/교통데이터 공모전/code/dataset/geo_dataset/'

In [50]:
import json

m = folium.Map(
    location=[37.5838699,127.0565831],
    zoom_start=10
)

with open(geo_path + 'seoul_municipalities_geo.json.txt', mode = 'rt', encoding = 'utf-8') as f:
    geo = json.loads(f.read())
    f.close()

folium.Choropleth(
    geo_data = geo,
    data = seoul_dlvr1,
    columns = ('시군구명', '배달상점광역시도명'),
    key_on = 'feature.properties.SIG_KOR_NM',
    fill_color = 'BuPu',
    legend_name = '요식업 점포 수'
).add_to(m)


for i in sc_region_seoul.index:
    sub_lat = sc_region_seoul.loc[i, '위도값']
    sub_lon = sc_region_seoul.loc[i, '경도값']
    
    title = sc_region_seoul.loc[i, 'Div_kor']
    
    color = 'green'
    if sc_region_seoul.loc[i, 'Div_kor'] == '폐업':
        color = 'red'
    
    folium.CircleMarker([sub_lat, sub_lon], color=color, radius = 3, tooltip = title).add_to(m)

m

In [51]:
kt_file_list

['서울시_직장인구.csv',
 'StartupOrClosing_Shop.csv',
 '서울시_동별_1인가구.txt',
 'dlvr_call.csv',
 'StartupOrClosing_Region.csv',
 '전체 데이터 설명.hwp',
 'popl.csv']

In [52]:
region_cols = ['인덱스','날짜','업종명','광역시도명','시군구명','읍면동','위도값','경도값','구분']
sc_region = pd.read_csv(kt_data_path + 'StartupOrClosing_Region.csv', header = 0, encoding = "cp949", names = region_cols)
sc_region.head()

seoul_pop = pd.read_csv(kt_data_path + '서울시_직장인구.csv', header = 0, encoding = "utf-8")
seoul_pop.head()

시군구명    동리명 길단위_상존인구  건물_상존인구  주거_인구 직장_인구 길단위_상존인구.1 건물_상존인구.1  주거_인구.1  \
0  종로구  청운효자동   14,357   27,286     52    44     13,195    24,233       52   
1  종로구    사직동   27,479  117,396     81   410     23,823    98,642       81   
2  종로구    삼청동    5,239   18,505     20    42      4,419    15,506       20   
3  종로구    부암동    4,855   27,923     45    19      4,767    26,355       45   
4  종로구    평창동    1,455    6,433     22    28      1,492     6,533       22   

  직장_인구.1 길단위_상존인구.2 건물_상존인구.2  주거_인구.2 직장_인구.2  
0      44     13,523    24,687       52      44  
1     410     23,520    96,592       81     410  
2      42      4,484    15,697       20      42  
3      19      4,784    26,419       45      19  
4      28      1,519     6,735       22      28